**Classifying “mask” and “without_mask” images.**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
main_dir = '/content/drive/MyDrive/faskmask/archive/New Masks Dataset'
train_dir = '/content/drive/MyDrive/facemask/archive/New Masks Dataset/Train'
test_dir = '/content/drive/MyDrive/facemask/archive/New Masks Dataset/Test'
valid_dir = '/content/drive/MyDrive/facemask/archive/New Masks Dataset/Validation'

train_mask_dir = '/content/drive/MyDrive/facemask/archive/New Masks Dataset/Test/Mask'
train_nomask_dir = '/content/drive/MyDrive/facemask/archive/New Masks Dataset/Test/Non Mask'


In [ ]:
print(train_dir)
print(train_mask_dir)

/content/drive/MyDrive/facemask/archive/New Masks Dataset/Train
/content/drive/MyDrive/facemask/archive/New Masks Dataset/Test/Mask


In [ ]:
train_mask_names = os.listdir(train_mask_dir)
print(train_mask_names[:10])

train_nomask_names = os.listdir(train_nomask_dir)
print(train_nomask_names[:10])

FileNotFoundError: ignored

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range = 0.2,
                                   rotation_range = 40,
                                   horizontal_flip = True
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )
valid_generator = validation_datagen.flow_from_directory(valid_dir,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )

In [ ]:
train_generator.class_indices

In [ ]:
train_generator.image_shape

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='SAME',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

NameError: ignored

In [ ]:
model.compile(Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    epochs = 30,
                    validation_data = valid_generator)

In [ ]:
test_loss , test_acc = model.evaluate(test_generator)
print('test acc :{} test loss:{}'.format(test_acc,test_loss))

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded= files.upload()
print(uploaded)
for f in uploaded.keys():
  img_path='/content/'+f
  img = image.load_img(img_path , target_size=(150,150))
  images = image.img_to_array(img)
  images=np.expand_dims(images,axis=0)
  prediction = model.predict(images)
  if prediction==0:
    print(f ,'mask is there')
  else:
    print(f,'nomask is present')

In [ ]:
model.save('saved_model.h5')